<a href="https://colab.research.google.com/github/PietroCaforio/research-biocv-proj/blob/dev/unimodal_ct_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

fatal: destination path 'research-biocv-proj' already exists and is not an empty directory.
M	data/processed/labels.txt
M	data/processed/overfit.txt
M	data/processed/train.txt
M	data/processed/val.txt
Already on 'dev'
Your branch is up to date with 'origin/dev'.


In [2]:
!cd research-biocv-proj && git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 652 bytes | 2.00 KiB/s, done.
From https://github.com/PietroCaforio/research-biocv-proj
   36cb07f..dce0f07  dev        -> origin/dev
Updating 36cb07f..dce0f07
Fast-forward
 data/unimodal.py    |   1 +
 preprocessing.ipynb | 586 +-----------------------------------------------------------------------
 2 files changed, 7 insertions(+), 580 deletions(-)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip -o /content/drive/MyDrive/processed57PatientsCPTAC-PDA.zip -d research-biocv-proj/data/

Archive:  /content/drive/MyDrive/processed57PatientsCPTAC-PDA.zip
   creating: research-biocv-proj/data/processed/CT/
  inflating: research-biocv-proj/data/processed/CT/C3L-00189.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00599.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00622.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00625.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-01032.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-01036.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-01687.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-01689.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-01703.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-02109.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-02112.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-02116.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-02118.npy  
  inflating

In [4]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

In [5]:
# Instantiate the dataset
dataset = UnimodalCTDataset(split='train', dataset_path = "research-biocv-proj/data/processed/")

# Check the length of the dataset
print(f"Dataset length: {len(dataset)}")

# Check the first few items in the dataset
for i in range(3):
    item = dataset[i]
    print(f"Item {i}:")
    print(f"  Patient ID: {item['patient_id']}")
    print(f"  Frame shape: {item['frame'].shape}")
    print(f"  Label: {item['label']}")

# Check if DataLoader works with the dataset
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Get a batch of data
batch = next(iter(dataloader))

# Check the batch
print(f"Batch patient IDs: {batch['patient_id']}")
print(f"Batch frame shape: {batch['frame'].shape}")
print(f"Batch labels: {batch['label']}")

# Move batch to device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset.move_batch_to_device(batch, device)
print(f"Batch moved to device: {device}")

Dataset length: 1508
Item 0:
  Patient ID: C3N-02295
  Frame shape: (3, 244, 244)
  Label: 0
Item 1:
  Patient ID: C3N-02295
  Frame shape: (3, 244, 244)
  Label: 0
Item 2:
  Patient ID: C3N-02295
  Frame shape: (3, 244, 244)
  Label: 0
Batch patient IDs: ['C3N-02295', 'C3N-01167', 'C3N-02971', 'C3N-00511']
Batch frame shape: torch.Size([4, 3, 244, 244])
Batch labels: tensor([0, 1, 0, 1])
Batch moved to device: cuda


### Train ResNet model

In [6]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [7]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "research-biocv-proj/data/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "research-biocv-proj/data/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [14]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        frames = batch['frame'].float().to(device)
        labels = batch['label'].long().to(device)

        optimizer.zero_grad()
        outputs = model(frames)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            frames = batch['frame'].float().to(device)
            labels = batch['label'].long().to(device)

            outputs = model(frames)
            loss = criterion(outputs.logits, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}")

Epoch 1, Loss: 1.0592137624820073
Validation Loss: 1.2411746762015603, Accuracy: 28.994082840236686
Epoch 2, Loss: 0.8851653796931108
Validation Loss: 1.2608912587165833, Accuracy: 26.035502958579883
Epoch 3, Loss: 0.5710877794772387
Validation Loss: 1.3317241126840764, Accuracy: 32.84023668639053
Epoch 4, Loss: 0.2592886191171904
Validation Loss: 1.6602858819744803, Accuracy: 34.319526627218934
Epoch 5, Loss: 0.0797419825491185
Validation Loss: 1.6466393281113019, Accuracy: 35.798816568047336
Epoch 6, Loss: 0.03332264575874433
Validation Loss: 1.742036103524945, Accuracy: 35.20710059171598
Epoch 7, Loss: 0.033611166678989925
Validation Loss: 1.9169988357885317, Accuracy: 32.248520710059175
Epoch 8, Loss: 0.021630875038681552
Validation Loss: 2.117519408633763, Accuracy: 37.27810650887574
Epoch 9, Loss: 0.02499554595851805
Validation Loss: 2.221602893688462, Accuracy: 39.053254437869825
Epoch 10, Loss: 0.02655997553180593
Validation Loss: 2.2758993872187356, Accuracy: 35.20710059171598

KeyboardInterrupt: 